In [11]:
import pandas as pd
import altair as alt
import numpy as np


file_name = 'inversión_visualización DISNEY.csv'
try:

    df = pd.read_csv(file_name, sep=';', encoding='latin-1')
except FileNotFoundError:
    file_name = '/content/inversión_visualización DISNEY.csv'
    df = pd.read_csv(file_name, sep=';', encoding='latin-1')


df.columns = df.columns.str.strip().str.replace('\r', '', regex=False)


TITULO_COL_NAME = df.columns[0]
INVERSION_COL_NAME = df.columns[3]
FECHA_COL_NAME = df.columns[2]

CLASIFICACION_COL_NAME = df.columns[4]



df['Inversión_Numerica'] = pd.to_numeric(
    df[INVERSION_COL_NAME].astype(str).str.strip().str.replace('"', '', regex=False).str.replace('.', '', regex=False).str.replace(',', '', regex=False),
    errors='coerce'
)
df['Año_Estreno'] = pd.to_numeric(
    df[FECHA_COL_NAME].astype(str).str.slice(0, 4),
    errors='coerce'
)

df_limpio = df.dropna(subset=['Año_Estreno', 'Inversión_Numerica', CLASIFICACION_COL_NAME]).copy()
df_limpio = df_limpio[df_limpio['Inversión_Numerica'] > 0]
df_limpio['Año_Estreno'] = df_limpio['Año_Estreno'].astype(int)


df_limpio['Tipo_Produccion_Detalle'] = df_limpio[CLASIFICACION_COL_NAME].str.strip()


df_limpio['Tipo_Produccion_Detalle'] = df_limpio['Tipo_Produccion_Detalle'].replace('Precuela', 'Secuela/Precuela')
df_limpio['Tipo_Produccion_Detalle'] = df_limpio['Tipo_Produccion_Detalle'].replace('Secuela', 'Secuela/Precuela')


alicia_condition = (
    df_limpio[TITULO_COL_NAME].str.lower().str.contains('alicia|alice', na=False) &
    (df_limpio['Año_Estreno'] == 2010)
)
df_limpio.loc[alicia_condition, 'Inversión_Numerica'] = 200000000



Y_LIM_MAX = 450000000
df_main = df_limpio[df_limpio['Inversión_Numerica'] <= Y_LIM_MAX].copy()


color_map_final = {
    'Secuela/Precuela': '#D41C16',      # ROJO
    'Live-Action': '#ffdf07',           # AMARILLO/DORADO
    'Original': '#1c5791'               # AZUL
}


tooltips = [
    alt.Tooltip(TITULO_COL_NAME, title='Título'),
    alt.Tooltip('Año_Estreno', title='Año', format='0f'),
    alt.Tooltip('Tipo_Produccion_Detalle', title='Tipo'),
    alt.Tooltip('Inversión_Numerica', title='Inversión', format='$,.0f')
]


base = alt.Chart(df_main).properties(
    title='Inversión por Año de Estreno y Tipo de Producción (Disney)',
    width=700,
    height=400
)


scatter_plot = base.mark_circle(size=100, opacity=0.8).encode(
    x=alt.X('Año_Estreno:O',
            title='Año de Estreno',
            axis=alt.Axis(
                labelAngle=-45,
                values=list(range(df_main['Año_Estreno'].min(), df_main['Año_Estreno'].max() + 1, 5))
            )
    ),
    y=alt.Y('Inversión_Numerica:Q',
            title='Inversión Total (USD)',
            scale=alt.Scale(domain=[0, Y_LIM_MAX * 1.05]),
            axis=alt.Axis(format='~s',
                          labelExpr="datum.value >= 1000000 ? format(datum.value / 1000000, ',.0f') + 'M' : format(datum.value, '$.0f')"
            )
    ),
    color=alt.Color('Tipo_Produccion_Detalle:N',
                    title='Tipo de Producción',
                    scale=alt.Scale(domain=list(color_map_final.keys()), range=list(color_map_final.values()))
    ),
    tooltip=tooltips
).interactive()


scatter_plot

alt.Chart(...)